In [37]:
import sys
# add to Path to import custom utils
import importlib


In [38]:
# call this to fetch changes in base code

import pre_processor

importlib.reload(pre_processor)

<module 'pre_processor' from '/Users/tsakalis/ntua/nestor/nestor_celvia/src/notebooks/pre_processor.py'>

In [31]:
import openslide
from pathlib import Path

# Open the SVS file

img_file = "TCGA-42-2593-01A-01-TS1.0aa136c4-90e2-49f3-adf8-73d334f9e6ec.svs"

full_path = Path(
    "/Users/tsakalis/Downloads/gdc_download_20241011_090627.261962/02dc3c23-e803-44e6-9651-ee8742372efd"
)

svs_image = openslide.OpenSlide(full_path / img_file)

# Get properties of the image
level = 0
dimensions = svs_image.level_dimensions[level]

region = svs_image.read_region((0, 0), level, dimensions)


In [39]:
pre_processor.extract_tiles(svs_image=svs_image,
                            tile_size=256,
                            slide_id="ssds")

[{'x': 0, 'y': 0, 'contains_tissue': False},
 {'x': 256, 'y': 0, 'contains_tissue': False},
 {'x': 512, 'y': 0, 'contains_tissue': False},
 {'x': 768, 'y': 0, 'contains_tissue': False},
 {'x': 1024, 'y': 0, 'contains_tissue': False},
 {'x': 1280, 'y': 0, 'contains_tissue': False},
 {'x': 1536, 'y': 0, 'contains_tissue': False},
 {'x': 1792, 'y': 0, 'contains_tissue': False},
 {'x': 2048, 'y': 0, 'contains_tissue': False},
 {'x': 2304, 'y': 0, 'contains_tissue': False},
 {'x': 2560, 'y': 0, 'contains_tissue': False},
 {'x': 2816, 'y': 0, 'contains_tissue': False},
 {'x': 3072, 'y': 0, 'contains_tissue': False},
 {'x': 3328, 'y': 0, 'contains_tissue': False},
 {'x': 3584, 'y': 0, 'contains_tissue': False},
 {'x': 3840, 'y': 0, 'contains_tissue': False},
 {'x': 4096, 'y': 0, 'contains_tissue': False},
 {'x': 4352, 'y': 0, 'contains_tissue': False},
 {'x': 4608, 'y': 0, 'contains_tissue': False},
 {'x': 4864, 'y': 0, 'contains_tissue': False},
 {'x': 5120, 'y': 0, 'contains_tissue': False}

In [4]:
dir_out = Path("../tiles/02dc3c23-e803-44e6-9651-ee8742372efd")

In [6]:
dir_out.mkdir(parents=True, exist_ok=True)

In [8]:
import numpy as np
import cv2
import openslide
from pathlib import Path
from skimage.morphology import closing, square
from skimage.filters import threshold_otsu
from skimage import img_as_ubyte
import h5py
from PIL import Image

# Open the SVS file
img_file = "TCGA-N9-A4Q7-01A-01-TS1.4A5EF75B-F961-40C1-89C6-CD00EF32291C.svs"
full_path = Path(
    "/Users/tsakalis/ntua/nestor/nestor_celvia/ovarian_tissue_data/9c9e1e7d-d651-4e24-a15b-019e720da792"
)
svs_image = openslide.OpenSlide(full_path / img_file)

patch_size = 256
level = 0
dimensions = svs_image.level_dimensions[level]


def create_tissue_mask(img: np.ndarray) -> np.ndarray:
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv_img)

    _, mask = cv2.threshold(s, 50, 255, cv2.THRESH_BINARY)

    mask = closing(mask, square(5))
    mask = img_as_ubyte(mask)

    return mask


def extract_patches(svs_image, patch_size, mask_threshold=210):
    slide_width, slide_height = svs_image.level_dimensions[level]

    patch_metadata = []
    i, j = 0, 0

    for y in range(0, slide_height - slide_height % patch_size, patch_size):
        i += 1
        for x in range(0, slide_width - slide_width % patch_size, patch_size):
            j += 1
            region = svs_image.read_region((x, y), level,
                                           (patch_size, patch_size))
            # continue
            region = np.array(region.convert("RGB"))

            mask = create_tissue_mask(region)

            flat_arr = np.array(region).flatten()

            bw_condition = flat_arr.shape[0] != (((flat_arr == 0).sum() +
                                                  (flat_arr > 210).sum()))

            if (np.mean(region) < mask_threshold) and bw_condition:
                Image.fromarray(region).save(dir_out / f"{i}_{j}.jpeg")

                patch_metadata.append({
                    "x": x,
                    "y": y,
                    "contains_tissue": True
                })

            else:
                patch_metadata.append({
                    "x": x,
                    "y": y,
                    "contains_tissue": False
                })

    return patch_metadata


def save_patches_to_hdf5(patch_metadata, output_path):
    with h5py.File(output_path, "w") as f:
        for i, patch in enumerate(patch_metadata):
            grp = f.create_group(f"patch_{i}")
            grp.attrs["x"] = patch["x"]
            grp.attrs["y"] = patch["y"]
            grp.attrs["contains_tissue"] = patch["contains_tissue"]


patch_metadata = extract_patches(svs_image, patch_size)
output_hdf5_path = "patch_metadata.hdf5"
save_patches_to_hdf5(patch_metadata, output_hdf5_path)

print(f"Saved patch metadata to {output_hdf5_path}")


Saved patch metadata to patch_metadata.hdf5


In [10]:
31 * 39

1209

In [8]:
9984 / 256

39.0

In [9]:
7936 / 256

31.0

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
svs_image.read_region?

In [ ]:
patch_size = 256

x, y = 0, 0

In [ ]:
level = 0

In [ ]:
patch_size = 256
# Read the full region (512x256)
reg1 = svs_image.read_region((3000, 3000), level, (patch_size, patch_size))

reg2 = svs_image.read_region((3000 + patch_size, 3000), level,
                             (patch_size, patch_size))


In [ ]:
whole_tiled_slide.shape

In [ ]:
plt.imshow(whole_tiled_slide, cmap='RGBA')


In [ ]:
whole_tiled_slide.shape

In [ ]:
plt.imshow(np.array(reg2))

In [ ]:
plt.imshow(np.array(reg1))

In [ ]:
display(reg1)
display(reg2)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(np.array(reg1)[:, :256, :])
plt.show()
plt.imshow(np.array(reg1)[:, 256:, :])

In [ ]:
display(reg1, reg2)

In [ ]:
slide_width, slide_height = svs_image.level_dimensions[level]
d = 256

In [ ]:
from itertools import product

grid = product(range(0, slide_height - slide_height % d, d),
               range(0, slide_width - slide_width % d, d))

In [ ]:
list(grid)

In [ ]:
import os
from PIL import Image


def tile(svs_image, dir_out, d):

    slide_width, slide_height = svs_image.level_dimensions[level]
    img = svs_image.read_region((0, 0), level, (patch_size, patch_size))

    grid = product(range(0, slide_height - slide_height % d, d),
                   range(0, slide_width - slide_width % d, d))
    for i, j in grid:
        box = (j, i, j + d, i + d)
        out = os.path.join(dir_out, f'{i}_{j}.jpeg')

        cropped_img = img.crop(box)
        cropped_img = cropped_img.convert('RGB')

        cropped_img.save(out)

In [ ]:
tile(svs_image, dir_out, 256)

In [ ]:
count = 0
for patch in patch_metadata:
    if patch['contains_tissue']:
        region = svs_image.read_region((patch['x'], patch['y']), level,
                                       (patch_size, patch_size))

        display(region)
        count += 1
        if count > 10:
            break

In [ ]:
region

In [ ]:
%timeit (np.random.random((100, 1000)) * np.random.random((100, 1)))

In [ ]:
region.show()

In [ ]:
import cv2
from PIL import Image
import numpy as np


def crop_and_resize(image, output_size=(256, 256)):
    # Open the image using OpenCV

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (15, 15), 0)

    # Use Otsu's thresholding to segment the tissue from the background
    _, thresh = cv2.threshold(blurred, 0, 255,
                              cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Find contours of the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
    print(contours)

    # If contours are found, get the bounding box around the largest contour (largest tissue area)
    if contours:
        # Get the largest contour
        largest_contour = max(contours, key=cv2.contourArea)

        # Get bounding box coordinates (x, y, width, height)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Crop the image using the bounding box
        cropped_image = image[y:y + h, x:x + w]

        # Resize the cropped image to the desired output size
        resized_image = cv2.resize(cropped_image, output_size)

        # Convert to PIL format for saving or further processing
        resized_pil_image = Image.fromarray(
            cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))

        return resized_pil_image
    else:
        # If no contours found, return the resized original image
        resized_image = cv2.resize(image, output_size)
        resized_pil_image = Image.fromarray(
            cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
        return resized_pil_image


# Example usage
# output_image = crop_and_resize('microscopy_image.jpg', output_size=(512, 512))
# output_image.save('resized_output_image.jpg')


In [ ]:
crop_and_resize(np.array(region))


In [ ]:
region.resize((1024, 1024), Image.Resampling.LANCZOS)

In [ ]:
region.show()

In [ ]:
# uterus tissue sample

img_file = "TCGA-N9-A4Q7-01A-01-TS1.4A5EF75B-F961-40C1-89C6-CD00EF32291C.svs"

full_path_uterus = Path(
    "/Users/tsakalis/Downloads/uterus_tissue/9c9e1e7d-d651-4e24-a15b-019e720da792"
)

In [ ]:
import openslide

# Open the SVS file
svs_image = openslide.OpenSlide(full_path_uterus / img_file)

# Get properties of the image
print("Image dimensions:", svs_image.dimensions)
print("Number of levels:", svs_image.level_count)
print("Level dimensions:", svs_image.level_dimensions)


In [ ]:
region = svs_image.read_region((0, 0), 3, (2453, 795))

In [ ]:
region

In [ ]:
from PIL import Image

In [ ]:
region.resize((1024, 1024), Image.Resampling.LANCZOS)

In [ ]:
import numpy as np

np.array(region).shape